In [ ]:
#Defined in Section 3.2.1 Train and test Split
#Function To Split a Dataset
#Split a dataset into a train and test set
sub train_test_split{
    my ($self, $dataset, %args) = (splice (@_, 0, 2), split =>0.6, @_);

    my $train_size = int($args{split} * $dataset->len);
    my $idx        = mx->nd->arange(stop => $dataset->len)->shuffle;
    my $train_idx  = $idx->slice(begin=>0, end=>$train_size);
    my $test_idx   = $idx->slice(begin=> $train_size, end=>$dataset->len);
    my $train      = mx->nd->take($dataset, $train_idx);
    my $test       = mx->nd->take($dataset, $test_idx);
    
    return $train, $test;
}

sml->add_to_class('train_test_split', \&{'train_test_split'});

In [ ]:


sub cross_validation_split{
    my ($self, $dataset, %args) = (splice (@_, 0, 2), n_folds=>10, @_);

    my @dataset_split;
    my $fold_size   = int($dataset->len / $args{n_folds});
    my $idx         = mx->nd->arange(stop=>$dataset->len)->shuffle;
    for my $i (0 .. $args{n_folds} -1){
        my $fold_idx = $idx->slice(begin=>$i * $fold_size, end=>($i +1) * $fold_size);
        push @dataset_split, mx->nd->take($dataset, $fold_idx);
    }

    return mx->nd->stack(@dataset_split, axis=>0);
}

sml->add_to_class('cross_validation_split', \&{'cross_validation_split'});

In [ ]:
sub count_labels{
    my(self, $dataset) = @_;
    my $Y = $dataset->slice_axis(axis=>1, begin=>-1, end=>$dataset->shape->[-1])
                      ->squeeze(axis=>1);
    my $num_classes = $Y->max->asscalar + 1;
    return mx->nd->one_hot($Y, $num_classes)->sum(axis=>0);
}

sml->add_to_class('count_labels', \&{'count_labels'});